In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,24663
2,Huelva,Confirmados PDIA 14 días,249
3,Huelva,Tasa PDIA 14 días,"48,521932303135415"
4,Huelva,Confirmados PDIA 7 días,116
5,Huelva,Total Confirmados,24883
6,Huelva,Curados,16963
7,Huelva,Fallecidos,308


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  24663.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6563.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 6849 personas en los últimos 7 días 

Un positivo PCR cada 2877 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,24663.0,116.0,249.0,513170.0,22.604595,48.521932,27.0
Huelva-Costa,15047.0,74.0,152.0,289548.0,25.557075,52.495614,18.0
Condado-Campiña,6906.0,40.0,93.0,156231.0,25.603113,59.527239,11.0
Ayamonte,1205.0,15.0,20.0,21104.0,71.076573,94.768764,5.0
Huelva (capital),6563.0,21.0,50.0,143837.0,14.599860,34.761570,4.0
Isla Cristina,1983.0,6.0,10.0,21393.0,28.046557,46.744262,4.0
San Bartolomé de la Torre,199.0,6.0,8.0,3761.0,159.532039,212.709386,4.0
Cartaya,955.0,12.0,28.0,20083.0,59.752029,139.421401,3.0
Palma del Condado (La),977.0,4.0,12.0,10801.0,37.033608,111.100824,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puebla de Guzmán,95.0,10.0,15.0,3092.0,323.415265,485.122898,1.0
Lucena del Puerto,157.0,6.0,11.0,3261.0,183.992640,337.319841,0.0
San Bartolomé de la Torre,199.0,6.0,8.0,3761.0,159.532039,212.709386,4.0
Cartaya,955.0,12.0,28.0,20083.0,59.752029,139.421401,3.0
Villablanca,176.0,1.0,4.0,2885.0,34.662045,138.648180,0.0
Palma del Condado (La),977.0,4.0,12.0,10801.0,37.033608,111.100824,2.0
Almonte,868.0,10.0,27.0,24507.0,40.804668,110.172604,0.0
Ayamonte,1205.0,15.0,20.0,21104.0,71.076573,94.768764,5.0
Condado-Campiña,6906.0,40.0,93.0,156231.0,25.603113,59.527239,11.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Aljaraque,925.0,1.0,8.0,21474.0,4.656794,37.254354,0.0,0.125000
Moguer,830.0,3.0,13.0,21867.0,13.719303,59.450313,1.0,0.230769
Villablanca,176.0,1.0,4.0,2885.0,34.662045,138.648180,0.0,0.250000
San Juan del Puerto,359.0,1.0,3.0,9411.0,10.625863,31.877590,0.0,0.333333
Palma del Condado (La),977.0,4.0,12.0,10801.0,37.033608,111.100824,2.0,0.333333
Almonte,868.0,10.0,27.0,24507.0,40.804668,110.172604,0.0,0.370370
Huelva (capital),6563.0,21.0,50.0,143837.0,14.599860,34.761570,4.0,0.420000
Cartaya,955.0,12.0,28.0,20083.0,59.752029,139.421401,3.0,0.428571
Condado-Campiña,6906.0,40.0,93.0,156231.0,25.603113,59.527239,11.0,0.430108
